In [1]:
import os
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw

# 文件夹路径
image_folder = '/Users/camus/Desktop/1'
mask_folder = '/Users/camus/Desktop/outputs'
output_folder = '/Users/camus/Desktop/2'

# 确保输出文件夹存在
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 遍历所有原始图像
for image_filename in os.listdir(image_folder):
    if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
        # 获取图像的完整路径
        image_path = os.path.join(image_folder, image_filename)
        
        # 获取对应的xml文件
        base_filename = os.path.splitext(image_filename)[0]  # 去除扩展名，保留文件名
        xml_filename = base_filename + '.xml'
        xml_path = os.path.join(mask_folder, xml_filename)

        if os.path.exists(xml_path):
            try:
                # 解析xml文件
                tree = ET.parse(xml_path)
                root = tree.getroot()

                # 打开图像文件
                image = Image.open(image_path)
                width = int(root.find('size/width').text)
                height = int(root.find('size/height').text)
                # 创建与原图相同大小的黑白图像（掩码）
                mask_image = Image.new('L', (width, height), 0)  # 使用灰度图 'L'

                draw = ImageDraw.Draw(mask_image)

                # 遍历XML文件中的所有对象
                for obj in root.findall('.//object'):
                    polygon = obj.find('.//polygon')
                    points = []
                    # 动态解析x1, y1 到 xN, yN 形式的坐标点
                    for i in range(1, 1000):  # 假设最多有1000个点，可以根据需要调整
                        x_element = polygon.find(f'x{i}')
                        y_element = polygon.find(f'y{i}')
                        if x_element is not None and y_element is not None:
                            x = float(x_element.text)
                            y = float(y_element.text)
                            points.append((x, y))
                        else:
                            break  # 终止循环，如果没有更多的点
                    
                    # 确保点列表有足够的点构成多边形
                    if len(points) >= 3:
                        # 确保点在图像范围内
                        points = [(max(0, min(width, x)), max(0, min(height, y))) for (x, y) in points]
                        # 绘制多边形
                        draw.polygon(points, outline=255, fill=255)
                    else:
                        print(f"警告: 多边形点数不足，无法绘制多边形: {points}")

                # 保存处理后的二值化图像
                output_image_path = os.path.join(output_folder, base_filename + '.png')
                mask_image.save(output_image_path)
                print(f'Saved: {output_image_path}')
            except Exception as e:
                print(f"处理 {image_filename} 时出错: {str(e)}")
        else:
            print(f'未找到对应的XML文件: {image_filename}')


Saved: /Users/camus/Desktop/2/4.png
Saved: /Users/camus/Desktop/2/1.png


In [4]:
import os
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw

# 图像和XML路径
image_path = '/Users/camus/Desktop/image-124_副本.png'
xml_path = '/Users/camus/Desktop/image-124_副本.xml'
output_path = '/Users/camus/Desktop/image-124_binary.png'

# 检查XML文件是否存在
if os.path.exists(xml_path):
    try:
        # 解析XML文件
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # 打开图像文件
        image = Image.open(image_path)
        width = int(root.find('size/width').text)
        height = int(root.find('size/height').text)

        # 创建与原图相同大小的黑白图像（掩码）
        mask_image = Image.new('L', (width, height), 0)  # 使用灰度图 'L'
        draw = ImageDraw.Draw(mask_image)

        # 遍历XML文件中的所有对象
        for obj in root.findall('.//object'):
            polygon = obj.find('.//polygon')
            points = []
            for i in range(1, 1000):  # 假设最多有1000个点，可以根据需要调整
                x_element = polygon.find(f'x{i}')
                y_element = polygon.find(f'y{i}')
                if x_element is not None and y_element is not None:
                    x = float(x_element.text)
                    y = float(y_element.text)
                    points.append((x, y))
                else:
                    break

            # 确保点列表有足够的点构成多边形
            if len(points) >= 3:
                points = [(max(0, min(width, x)), max(0, min(height, y))) for (x, y) in points]
                draw.polygon(points, outline=255, fill=255)
            else:
                print(f"警告: 多边形点数不足，无法绘制多边形: {points}")

        # 保存处理后的二值化图像
        mask_image.save(output_path)
        print(f'已保存: {output_path}')
    except Exception as e:
        print(f"处理图像时出错: {str(e)}")
else:
    print(f'未找到对应的XML文件: {xml_path}')


已保存: /Users/camus/Desktop/image-124_binary.png
